# **Fit GLM-HMM to individual DMDM data**
---
After fittinng GLM to all animals, we can finally fit GLM-HMM to individual animals in the dataset. Just like global GLM-HMM, we only use outcomes `y` as a dependent variable.

## **HPC setting**
Ashwood's original script is written in python scirpts. Here, we rewrite it in Jupyter to make it more user-friendly to run on HPC with `dask`. [This](https://github.com/pierreglaser/hpc-tutorial/tree/main) is very useful resource to get familiar with `dask`.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# allocate the computing resources
from dask_jobqueue import SLURMCluster
from distributed import Client
from joblib import Memory, Parallel, delayed, parallel_backend
from threadpoolctl import threadpool_limits
from tqdm import tqdm

cluster = SLURMCluster(
    workers=0,      # create the workers "lazily" (upon cluster.scal)
    memory='64g',   # amount of RAM per worker
    processes=1,    # number of execution units per worker (threads and processes)
    cores=1,        # among those execution units, number of processes
    walltime="240:00:00",
    job_extra_directives=[
        "--gres=gpu:1",
        "-p gpu",
    ], # the only way to add GPUs
    local_directory='/ceph/mrsic_flogel/public/projects/SuKuMiLo_20220107_DMDM_Human/Jobs', # set your path to save log
    log_directory='/ceph/mrsic_flogel/public/projects/SuKuMiLo_20220107_DMDM_Human/Jobs' # set your path to save log /nfs/nhome/live/skuroda/jobs
)   

memory = Memory('/nfs/nhome/live/skuroda/joblib-cache') # set your path

n = 32
cluster.scale(n)
client = Client(cluster)
print(client.dashboard_link)

/nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/dask_jobqueue/core.py:17: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile, get_ip_interface
/nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/dask_jobqueue/core.py:17: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile, get_ip_interface
/nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/dask_jobqueue/core.py:17: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile, get_ip_interface
/nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/dask_jobqueue/

http://192.168.234.51:8787/status


## **Fit GLM-HMM to individual animals using MAP**
---
We use MAP instead of MLE for EM algorithm here. To select the hyperparameters σ and α governing the prior, we performed a grid search for σ ∈ {0.5, 1, 1.5,  2} and α ∈ {2} and later selected the hyperparameters that resulted in the best performance **within training dataset**. (Don't use the ones with the best performance on a held-out validation set, as it can overfit hyperparameters.) Hence, we first fit GLM-HMM to tune the parameter and repeat the fitting process to retrieve the best GLM results for each state/animal. The first grid search requires quite a long time.

In [3]:
# ------- load modules -------
import autograd.numpy as np
import numpy as onp
import autograd.numpy.random as npr
import sys
import itertools

from glm_hmm_utils import fit_glm_hmm
sys.path.append('../') # a lazy trick to search parent dir
# https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im
from data_io import get_file_dir, load_session_fold_lookup, load_data, load_animal_list, load_glmhmm_data, load_global_best_params
from data_labels import create_abort_mask, partition_data_by_session

from functools import partial
from collections import OrderedDict

In [4]:
# ------- setup variables -------
dname = 'dataAllMiceTraining'
C = 3  # number of output types/categories
D = 1  # data (observations) dimension
prior_sigma = [1, 2, 3]
transition_alpha = [1, 2] # alpha = 1 performs MLE
K_vals = [1, 2, 3, 4]
N_initializations = 2 #20
num_folds_training = num_folds_tuning = 5

nested_outcome = OrderedDict() # define nested structure for behavioral outcomes
nested_outcome["Baseline"] = [2]
nested_outcome["Change"] = [0, 1]

cluster_job_arr = []
for K in K_vals:
    for i in range(num_folds_training):
        for ii in range(num_folds_tuning):
            for j in range(N_initializations):
                    for sigma in prior_sigma:
                        for alpha in transition_alpha:
                            cluster_job_arr.append([sigma, alpha, K, i, ii, j])

N_em_iters = 600  # number of EM iterations
global_fit = False
paramter_tuning = True

In [5]:
# ------- setup path and load data -------
data_2_dir =  get_file_dir().parents[1] / "data" / "dmdm" / dname / 'data_for_cluster' / "data_by_animal"
# Create directory for results:
try: 
    results_2_dir = get_file_dir().parents[1] / "results" / "dmdm_individual_fit" / dname
except:
    raise FileNotFoundError('Run GLM first to initialize parameters')

#  read in data and train/test split
animal_list = load_animal_list(data_2_dir / 'animal_list.npz')
cluster_job_arr_with_animal = list(itertools.product(cluster_job_arr, animal_list))

print('Total number of jobs: {}'.format(len(cluster_job_arr_with_animal)))
print('Animals for individual fitting: {}'.format(animal_list))

Total number of jobs: 16800
Animals for individual fitting: ['M_AK001' 'M_AK004' 'M_AK005' 'M_AK008' 'M_IO125' 'M_IO127' 'M_IO128'
 'M_IO132' 'M_IO135' 'M_IO136' 'M_IO137' 'M_IO138' 'M_ML007' 'M_ML008']


In [6]:
def fit_GLMHMM_y_separetely(data_2_dir, results_2_dir, paramter_tuning, cluster_job_arr_with_animal):
    animal = cluster_job_arr_with_animal[1]
    print(animal)

    allparams = cluster_job_arr_with_animal[0]
    prior_sigma = allparams[0]
    transition_alpha = allparams[1]
    HMM_params = [int(p) for p in allparams[2:]]


    animal_file = data_2_dir / (animal + '_processed.npz')
    session_fold_lookup_table = load_session_fold_lookup(
        data_2_dir / (animal + '_session_fold_lookup.npz'))
    this_results_dir = results_2_dir / animal

    inpt_y, inpt_rt, y, session, rt, stim_onset = load_data(animal_file)

    fit_GLMHMM_y(inpt_y, y, session, 
                 session_fold_lookup_table, 
                 global_fit,
                 paramter_tuning,
                 transition_alpha,
                 prior_sigma,
                 this_results_dir,
                 HMM_params)

def fit_GLMHMM_y(inpt_y, y, session, 
                 session_fold_lookup_table, 
                 global_fit,
                 paramter_tuning,
                 transition_alpha,
                 prior_sigma,
                 results_dir,
                 params):
    
    [K, fold_training, fold_tuning, iter] = params

    # Append a column of ones to inpt to represent the bias covariate:
    inpt_y = np.hstack((inpt_y, np.ones((len(inpt_y),1))))
    y = y.astype('int')
    # Identify violations for exclusion:
    abort_idx = np.where(y == 3)[0]
    nonviolation_idx, mask = create_abort_mask(abort_idx, inpt_y.shape[0])

    #  GLM weights to use to initialize GLM-HMM
    if global_fit == True:
        raise NotImplementedError('This notebook only runs individual fitting')
    else:
        init_param_file = data_2_dir.parents[0] / 'best_global_params' / ('best_params_GLM_HMM_y_K_' + str(K) + '.npz')
        # fold does not matter here.

    # Create save directory for this initialization/fold combination:
    if paramter_tuning:
        saving_directory = results_dir / ("GLM_HMM_y_K_" + str(K)) / ("fold_" + str(fold_training)) / ("tuningfold_" + str(fold_tuning)) / ('iter_' + str(iter))
    else:
        saving_directory = results_dir / ("GLM_HMM_y_K_" + str(K)) / ("fold_" + str(fold_training)) / ('iter_' + str(iter))
    saving_directory.mkdir(parents=True, exist_ok=True)

    launch_glm_hmm_job(inpt_y,
                       y,
                       session,
                       mask,
                       session_fold_lookup_table,
                       K,
                       D,
                       C,
                       N_em_iters,
                       transition_alpha,
                       prior_sigma,
                       fold_training,
                       fold_tuning,
                       paramter_tuning,
                       iter,
                       global_fit,
                       init_param_file,
                       saving_directory)


def launch_glm_hmm_job(inpt, y, session, mask, session_fold_lookup_table, K, D,
                       C, N_em_iters, transition_alpha, prior_sigma, fold_training, 
                       fold_tuning, paramter_tuning, iter, global_fit, init_param_file, save_directory):
    sys.stdout.flush()
    if paramter_tuning:
        split_loc = np.logical_and(session_fold_lookup_table[:, 1] != fold_training, session_fold_lookup_table[:, 2] != fold_tuning)
        sessions_to_keep = session_fold_lookup_table[split_loc, 0]
    else:
        sessions_to_keep = session_fold_lookup_table[np.where(
            session_fold_lookup_table[:, 1] != fold_training), 0]

    idx_this_fold = [str(sess) in sessions_to_keep for sess in session]
    this_inpt, this_y, this_session, this_mask = inpt[idx_this_fold, :], \
                                                 y[idx_this_fold, :], \
                                                 session[idx_this_fold], \
                                                 mask[idx_this_fold, :]
    
    # Only do this so that errors are avoided - these y values will not
    # actually be used for anything (due to violation mask)
    this_y[np.where(this_y == 3), :] = 2
    
    inputs, datas, masks = partition_data_by_session(
        this_inpt, this_y, this_mask, this_session)
    # Read in GLM fit if global_fit = True:
    if global_fit == True:
         raise NotImplementedError('This notebook only runs individual fitting')
    else:
        params_for_initialization = load_global_best_params(init_param_file)
    M = this_inpt.shape[1]

    npr.seed(iter)
    fit_glm_hmm(datas,
                inputs,
                masks,
                K,
                D,
                M,
                C,
                N_em_iters,
                transition_alpha,
                prior_sigma,
                global_fit,
                False,
                0,
                params_for_initialization,
                save_title=save_directory / ('GLM_HMM_y_raw_parameters_itr_' + str(iter))
                )

fit_GLMHMM_separetely_eachparam = partial(fit_GLMHMM_y_separetely, data_2_dir, results_2_dir, 
                                           paramter_tuning)        
fit_GLMHMM_separetely_eachparam_cached = memory.cache(fit_GLMHMM_separetely_eachparam)

In [7]:
with Client(cluster) as client: # upload local functions to each worker. They cannot read them with sys.append or sys.insert.
    client.wait_for_workers(n)
    client.upload_file(str(get_file_dir() / 'data_io.py'))
    client.upload_file(str(get_file_dir() / 'data_labels.py'))

In [8]:
%%time

with threadpool_limits(limits=1, user_api='blas'):
    with parallel_backend('dask', wait_for_workers_timeout=120):
        Parallel(verbose=100)(
            delayed(fit_GLMHMM_separetely_eachparam)(allparams) for allparams in cluster_job_arr_with_animal
            )

[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   13.2s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done  15 tasks      |

In [10]:
# Once finished, shut down the cluster and the client.
memory.clear(warn=False)
cluster.close()
client.close()

## **MAP hyperparameter tuning**
---

We first check the output files and save the best initialization results within hyperparameter-tuning-cross-validation. We then find the best sigma and alpha hyperparameters by averaged performance of all folds for each state `K`, test fold `k`, and animal `animal`.

In [11]:
from kfold_cv import KFoldCV
from data_io import load_data, load_session_fold_lookup, load_glmhmm_data, load_best_map_params, \
                     load_cv_arr, get_file_name_for_best_glmhmm_fold
from data_postprocessing_utils import calculate_state_permutation
import json

model = 'GLM_HMM_y'
animal_and_training_folds = list(itertools.product(animal_list, range(num_folds_training)))
print('Animals for individual fitting: {}'.format(animal_list))

In /nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two mi

Animals for individual fitting: ['M_AK001' 'M_AK004' 'M_AK005' 'M_AK008' 'M_IO125' 'M_IO127' 'M_IO128'
 'M_IO132' 'M_IO135' 'M_IO136' 'M_IO137' 'M_IO138' 'M_ML007' 'M_ML008']


In [12]:
out_params = np.zeros((len(K_vals)*num_folds_training*len(animal_list), 5), dtype='<U32') # sigma, alpha, K, fold, and animal
idx_count = 0

for idx, (animal, fold_training) in enumerate(animal_and_training_folds):
    print(animal)
    this_results_dir = results_2_dir / animal

    animal_file = data_2_dir / (animal + '_processed.npz')
    session_fold_lookup_table = load_session_fold_lookup(
        data_2_dir / (animal + '_session_fold_lookup.npz'))
    inpt_y, inpt_rt, y, session, rt, stim_onset = load_data(animal_file)

    KFCV = KFoldCV(model,num_folds_tuning, K_vals, global_fit, 
                   transition_alpha,prior_sigma, None,
                   this_results_dir)
    out = KFCV.save_best_MAPparams(inpt_y, inpt_rt, y, session, rt, stim_onset,
                                    session_fold_lookup_table, fold_training, C, 
                                    outcome_dict=nested_outcome, save_output=True)

    out_arr = np.array(out)[:,[2,1,0]] # sort

    out_arr_fold_animal = np.hstack([out_arr, 
                                      np.repeat(fold_training, len(K_vals)).T[:, None], 
                                      np.repeat(animal, len(K_vals)).T[:, None]])
                                    
    out_params[idx_count:idx_count+len(K_vals),:] = out_arr_fold_animal
    idx_count += len(K_vals)

np.savez(data_2_dir / "best_params_model_{}.npz".format(model), out_params)
print("Saved all the MAP params!")

M_AK001
Retrieving best iter results for model = GLM_HMM_y; fold = 0; num_folds = 5


100%|██████████| 5/5 [01:00<00:00, 12.09s/it]


Calculating best MAP parameters...
M_AK001
Retrieving best iter results for model = GLM_HMM_y; fold = 1; num_folds = 5


100%|██████████| 5/5 [00:51<00:00, 10.26s/it]


Calculating best MAP parameters...
M_AK001
Retrieving best iter results for model = GLM_HMM_y; fold = 2; num_folds = 5


...

Calculating best MAP parameters...
M_ML008
Retrieving best iter results for model = GLM_HMM_y; fold = 3; num_folds = 5


100%|██████████| 5/5 [01:18<00:00, 15.73s/it]


Calculating best MAP parameters...
M_ML008
Retrieving best iter results for model = GLM_HMM_y; fold = 4; num_folds = 5


100%|██████████| 5/5 [01:18<00:00, 15.69s/it]


Calculating best MAP parameters...
Saved all the MAP params!


## **Refitting GLM-HMM to individual animals with best MAP hyperparameters**
---

We fit GLM-HMM to separate animals again, but using the best MAP hyperparams we just found.

In [13]:
paramter_tuning = False
fold_dummy = 100

raw_params = load_best_map_params(data_2_dir / "best_params_model_{}.npz".format(model))
params_noanimal = raw_params[:,0:-1].astype('float32')

In [14]:
np.array([l[0] for l in raw_params])

array(['3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '2', '2', '3',
       '3', '2', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '1', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '2', '2', '3', '3', '1', '1', '3', '3', '1', '1',
       '3', '3', '3', '3', '3', '3', '1', '1', '3', '3', '1', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '2', '2',
       '3', '3', '3', '3', '2', '3', '3', '2', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '2', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3

In [24]:
cluster_job_arr_2 = []
for i in range(len(params_noanimal)):
        for j in range(N_initializations):
                        param_i = [params_noanimal[i,0], 
                                   params_noanimal[i,1],
                                   params_noanimal[i,2],
                                   params_noanimal[i,3],
                                   fold_dummy, j]
                        cluster_job_arr_2.append((param_i, raw_params[:,-1].tolist()[i]))

print('Total number of jobs: {}'.format(len(cluster_job_arr_2)))
print('Animals for individual fitting: {}'.format(animal_list))

Total number of jobs: 560
Animals for individual fitting: ['M_AK001' 'M_AK004' 'M_AK005' 'M_AK008' 'M_IO125' 'M_IO127' 'M_IO128'
 'M_IO132' 'M_IO135' 'M_IO136' 'M_IO137' 'M_IO138' 'M_ML007' 'M_ML008']


In [25]:
fit_GLMHMM_separetely_eachparam_2 = partial(fit_GLMHMM_y_separetely, data_2_dir, results_2_dir, 
                                           paramter_tuning)        
fit_GLMHMM_separetely_eachparam_2_cached = memory.cache(fit_GLMHMM_separetely_eachparam_2)

In [26]:
with Client(cluster) as client: # upload local functions to each worker. They cannot read them with sys.append or sys.insert.
    client.wait_for_workers(n)
    client.upload_file(str(get_file_dir() / 'data_io.py'))
    client.upload_file(str(get_file_dir() / 'data_labels.py'))

In [27]:
%%time

with threadpool_limits(limits=1, user_api='blas'):
    with parallel_backend('dask', wait_for_workers_timeout=120):
        Parallel(verbose=100)(
            delayed(fit_GLMHMM_separetely_eachparam_2)(allparams) for allparams in cluster_job_arr_2
            )

[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.7s


distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 11.6min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed: 12.9min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 15.5min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed: 15.6min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 17.0min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 17.0min
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed: 17.1min
[Paralle

In [28]:
# Once finished, shut down the cluster and the client.
memory.clear(warn=False)
cluster.close()
client.close()

/nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/dask_jobqueue/core.py:321: FutureWarning: ignoring was deprecated in version 2021.06.1 and will be removed in a future release. Please use contextlib.suppress from the standard library instead.
  with ignoring(RuntimeError):  # deleting job when job already gone
/nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/dask_jobqueue/core.py:321: FutureWarning: ignoring was deprecated in version 2021.06.1 and will be removed in a future release. Please use contextlib.suppress from the standard library instead.
  with ignoring(RuntimeError):  # deleting job when job already gone
/nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/dask_jobqueue/core.py:321: FutureWarning: ignoring was deprecated in version 2021.06.1 and will be removed in a future release. Please use contextlib.suppress from the standard library instead.
  with ignoring(RuntimeError):  # deleting job when job already gon

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


## **Posthoc data processing**
---

In [29]:
from kfold_cv import KFoldCV
from data_io import load_data, load_session_fold_lookup, load_glmhmm_data, load_cv_arr, get_file_name_for_best_glmhmm_fold, get_best_map_params
from data_postprocessing_utils import calculate_state_permutation
sys.path.append('../../../3_make_figures/dmdm/')
from plot_model_perform import plot_states, plot_model_comparison, plot_state_occupancy
import json

model = 'GLM_HMM_y'
labels_for_plot_y = ['CSize', 'TempExp', 'HzrdBlck', 'PrevHit +1', 'PrevFA +1', 'bias']
print('Animals for individual fitting: {}'.format(animal_list))

Animals for individual fitting: ['M_AK001' 'M_AK004' 'M_AK005' 'M_AK008' 'M_IO125' 'M_IO127' 'M_IO128'
 'M_IO132' 'M_IO135' 'M_IO136' 'M_IO137' 'M_IO138' 'M_ML007' 'M_ML008']


In [30]:
raw_params = load_best_map_params(data_2_dir / "best_params_model_{}.npz".format(model))

for animal in animal_list:
    print(animal)
    this_results_dir = results_2_dir / animal

    animal_file = data_2_dir / (animal + '_processed.npz')
    session_fold_lookup_table = load_session_fold_lookup(
        data_2_dir / (animal + '_session_fold_lookup.npz'))
    inpt_y, inpt_rt, y, session, rt, stim_onset = load_data(animal_file)

    KFCV = KFoldCV(model, num_folds_training, K_vals, global_fit, 
                   transition_alpha, prior_sigma,None,
                   this_results_dir, animal=animal)
    KFCV.save_best_iter(inpt_y, inpt_rt, y, session, rt, stim_onset,
                         session_fold_lookup_table, C, outcome_dict=nested_outcome,
                         map_params=raw_params)


M_AK001
Retrieving best iter results for model = GLM_HMM_y; num_folds = 5


100%|██████████| 5/5 [00:38<00:00,  7.65s/it]


Best init saved!
M_AK004
Retrieving best iter results for model = GLM_HMM_y; num_folds = 5


100%|██████████| 5/5 [00:38<00:00,  7.62s/it]


Best init saved!
M_AK005
Retrieving best iter results for model = GLM_HMM_y; num_folds = 5


...

Best init saved!
M_ML007
Retrieving best iter results for model = GLM_HMM_y; num_folds = 5


100%|██████████| 5/5 [00:27<00:00,  5.58s/it]


Best init saved!
M_ML008
Retrieving best iter results for model = GLM_HMM_y; num_folds = 5


100%|██████████| 5/5 [00:58<00:00, 11.71s/it]

Best init saved!


In [31]:
for animal in animal_list:
    print(animal)
    this_results_dir = results_2_dir / animal
    saving_directory = data_2_dir / ("best_params_" + animal)
    saving_directory.mkdir(parents=True, exist_ok=True)

    animal_file = data_2_dir / (animal + '_processed.npz')
    session_fold_lookup_table = load_session_fold_lookup(
        data_2_dir / (animal + '_session_fold_lookup.npz'))
    inpt_y, inpt_rt, y, session, rt, stim_onset = load_data(animal_file)

    cvbt_folds_model = load_cv_arr(this_results_dir / "cvbt_folds_model_{}.npz".format(model))
    cvbt_train_folds_model = load_cv_arr(this_results_dir / "cvbt_train_folds_model_{}.npz".format(model))

    for model_idx, K in enumerate(K_vals):
        print("K = " + str(K))
        with open(this_results_dir / "best_init_cvbt_dict_{}.json".format(model), 'r') as f:
            best_init_cvbt_dict = json.load(f)

        # Get the normalized log likelihood corresponding to the best initialization for
        # given K, alpha, and sigma

        raw_file_bestmap, _ = get_file_name_for_best_glmhmm_fold(
            cvbt_folds_model, model_idx, K, 
            overall_dir = this_results_dir, best_init_cvbt_dict = best_init_cvbt_dict, 
            model = model, fname_header = 'GLM_HMM_y_raw_parameters_itr_',
            global_fit=False, map_params=raw_params, animal=animal)
        hmm_params, lls, _, _, _= load_glmhmm_data(raw_file_bestmap)

        # Calculate permutation
        init_state_dist, log_transition_matrix, weight_vectors, permutation = \
            calculate_state_permutation(hmm_params, K)

        if K == 1:
            best_params = weight_vectors
        elif K > 1:
            best_params = [[init_state_dist], [log_transition_matrix], weight_vectors]

        plot_states(weight_vectors,
                    log_transition_matrix,
                    saving_directory,
                    K,
                    save_title='best_params_' + model + '_K_',
                    labels_for_plot=labels_for_plot_y)
        if K > 1:
            plot_state_occupancy(inpt_y, inpt_rt, y, session, rt, stim_onset,
                                    K, hmm_params, animal,
                                    saving_directory,
                                    save_title='best_state_occpancy_' + model + '_K_')
        np.savez(saving_directory / ('best_params_' + model + '_K_' + str(K) + '.npz'),
                best_params)
                
    plot_model_comparison(cvbt_folds_model,
                           cvbt_train_folds_model,
                           global_fit,
                           K_vals,
                           saving_directory)

M_AK001
K = 1
K = 2


/nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


K = 3


/nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


K = 4


...

/nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


K = 3


/nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


K = 4


/nfs/nhome/live/skuroda/.conda/envs/glmhmm/lib/python3.7/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)
../../../3_make_figures/dmdm/plot_model_perform.py:201: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=('ci', 68)` for the same effect.

  lw=4)
../../../3_make_figures/dmdm/plot_model_perform.py:213: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=('ci', 68)` for the same effect.

  lw=4)
